In [2]:
import dash
from dash import html, dcc, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd
import os
# from sentiment_analysis import analyze_sentiment  # Assume this is a function you've defined to analyze sentiment




In [8]:
import dash
from dash import dcc, html, Input, Output, State, callback
import dash_bootstrap_components as dbc
import dash_table
import plotly.express as px
import pandas as pd
import os
from wordcloud import WordCloud
import base64
from io import BytesIO
import torch

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Twitter Sentiment Analysis Dashboard"),
            dcc.Upload(
                id='upload-data',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Files')
                ]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                    'textAlign': 'center', 'margin': '10px'
                },
                multiple=False  
            ),
            dbc.Button("Analyze Sentiment with BERT", id="btn-analyze", color="success"),
            html.Br(),
            html.Br(),
            dbc.Button("Display Visualization", id="btn-display", color="info")
        ], width=4),
        dbc.Col([
            dcc.Graph(id='sentiment-chart'),
            html.Img(id='visualization'),
            dash_table.DataTable(id='table')
        ], width=8)
    ])
])

@callback(
    Output('table', 'data'),
    Output('table', 'columns'),
    Input('upload-data', 'contents'),
    prevent_initial_call=True
)
def update_output(contents):
    if contents:
        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        try:
            df = pd.read_csv(BytesIO(decoded))
            return df.head(10).to_dict('records'), [{"name": i, "id": i} for i in df.columns]
        except Exception as e:
            print(e)
            return [], []

@callback(
    Output('sentiment-chart', 'figure'),
    Input('btn-analyze', 'n_clicks'),
    State('table', 'data'),
    prevent_initial_call=True
)
def update_sentiment_output(n_clicks, rows):
    if n_clicks and rows:
        df = pd.DataFrame(rows)
        # df['Sentiment'] = predict_sentiment(df['content'].tolist())
        fig = px.bar(df['Sentiment'].value_counts(), title="Sentiment Analysis Results")
        return fig

@callback(
    Output('visualization', 'src'),  
    Input('btn-display', 'n_clicks'),
    State('table', 'data'),
    prevent_initial_call=True
)
def update_visualization(n_clicks, rows):
    if n_clicks and rows:
        df = pd.DataFrame(rows)
        text = " ".join(review for review in df.content)
        wordcloud = WordCloud(background_color='white').generate(text)
        img = BytesIO()
        wordcloud.to_image().save(img, format='PNG')
        return "data:image/png;base64,{}".format(base64.b64encode(img.getvalue()).decode())

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/np/rvthtbk913nd1z49b1gr5tdc0000gn/T/ipykernel_93466/2478652828.py:4: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`

